# Data Preparation

Prepare train/val/test splits for coconut tree detection.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-object-detection-on-aerial-imagery/blob/master/notebooks/experiment.ipynb)

In [1]:
# !pip install dl4cv_oda

In [2]:
import requests
import geopandas as gpd
from pathlib import Path
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)

In [3]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded OSM data")

if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Cleaned {count} trees")

if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Downloaded tiles")

In [4]:
if not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"Clipped {stats['total_trees']} trees to {stats['processed']} tiles")
    
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Converted to YOLO format")
    
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42)
    print(f"Split: train={train_count}, val={val_count}, test={test_count}")
    
    config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
    print(f"Config: {config_file}")

print("Data preparation complete")

Data preparation complete


In [5]:
from datetime import datetime
import pandas as pd
from ultralytics import YOLO, RTDETR
import torch
import yaml

## Configuration

In [6]:
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)


# for training
SEED = 64
IMG_SIZE = 256
EPOCHS = 200 
PATIENCE = 15
BATCH = 16

# for hyper param tuning
TUNE_MODELS = False
TUNE_ITERATIONS = 10
TUNE_EPOCHS = 20

MODELS = [
    {"name": "yolov8l", "weights": "yolov8l.pt"},
    {"name": "yolo12l", "weights": "yolo12l.pt"},
    {"name": "rtdetr-l", "weights": "rtdetr-l.pt"},
    # {"name": "yolo_nas_l", "weights": "yolo_nas_l.pt"},
    
]
EXPERIMENT_NAME = "default_run"

torch.manual_seed(SEED)
exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
exp_id = f"{EXPERIMENT_NAME}_{exp_id}"
print(f"Experiment: {exp_id}")
print(f"Models: {[m['name'] for m in MODELS]}")

Experiment: default_run_20260111_200230
Models: ['yolov8l', 'yolo12l', 'rtdetr-l']


## Hyperparameter Tuning & Training 

In [ ]:

results = []

for model_cfg in MODELS:
    name = model_cfg['name']

    model = RTDETR(model_cfg['weights']) if 'rtdetr' in name.lower() else YOLO(model_cfg['weights'])
    
    
    if TUNE_MODELS:
        tuning_name = f"{exp_id}_{name}_tune"
            
        print(f"\nTuning {name} for {TUNE_ITERATIONS} iterations")
        model.tune(
            data=str(YOLO_DIR / "config.yaml"),
            name = tuning_name,
            epochs=TUNE_EPOCHS,
            iterations=TUNE_ITERATIONS,
            imgsz=IMG_SIZE,
            plots=False,
            save=False,
            val=True
        )

        print(f"{name} tuning complete. Best hyperparameters saved to runs/detect/{tuning_name}/best_hyperparameters.yaml")
    

    tuned_params = {}
    best_cfg_path = Path(f"runs/detect/{exp_id}_{name}_tune/best_hyperparameters.yaml")

    if best_cfg_path.exists():
        with open(best_cfg_path, 'r') as f:
            tuned_params = yaml.safe_load(f)
        if 'close_mosaic' in tuned_params:
            tuned_params['close_mosaic'] = int(tuned_params['close_mosaic'])


    model.train(
        data=str(YOLO_DIR / "config.yaml"),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        patience=PATIENCE,
        batch=BATCH,
        seed=SEED,
        name=f"{exp_id}_{name}",
        project="runs",
        plots=False,
        verbose=False,
        **tuned_params
    )
    
    val_metrics = model.val(split='val', verbose=False)
    test_metrics = model.val(split='test', verbose=False)
    
    exp_results_dir = RESULTS_DIR / exp_id
    exp_results_dir.mkdir(exist_ok=True)
    
    results_csv = Path(f"runs/{exp_id}_{name}/results.csv")
    if results_csv.exists():
        import shutil
        shutil.copy(results_csv, exp_results_dir / f"{name}_results.csv")
    
    if best_cfg_path.exists():
        import shutil
        shutil.copy(best_cfg_path, exp_results_dir / f"{name}_best_hyperparameters.yaml")
    
    val_p, val_r = float(val_metrics.box.mp), float(val_metrics.box.mr)
    test_p, test_r = float(test_metrics.box.mp), float(test_metrics.box.mr)
    
    val_f1 = 2 * (val_p * val_r) / (val_p + val_r + 1e-6)
    test_f1 = 2 * (test_p * test_r) / (test_p + test_r + 1e-6)
    
    results.append({
        'model': name,
        'tuned': TUNE_MODELS,
        'val_precision': val_p,
        'val_recall': val_r,
        'val_f1': val_f1,
        'val_map50': float(val_metrics.box.map50),
        'test_precision': test_p,
        'test_recall': test_r,
        'test_f1': test_f1,
        'test_map50': float(test_metrics.box.map50),
    })
    
    print(f"{name}: val_f1={val_f1:.4f}, test_f1={test_f1:.4f}, test_map50={results[-1]['test_map50']:.4f}")



New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=default_run_20

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/200      9.07G      1.848     0.4849     0.7313        496        256: 100% ━━━━━━━━━━━━ 20/20 3.5it/s 5.7s0.2s
      1/200      9.07G      1.848     0.4849     0.7313        496        256: 100% ━━━━━━━━━━━━ 20/20 3.5it/s 5.7s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.4it/s 1.2s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.4it/s 1.2s
                   all         93       2846     0.0613       0.52     0.0525     0.0152
                   all         93       2846     0.0613       0.52     0.0525     0.0152

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      2/200      4.98G      1.438     0.3973     0.3429        519        256: 5% ╸─────────── 1/20 1.9it/s 0.3s<10.2s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/200      6.23G      1.153     0.4962     0.2515        790        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
      2/200      6.23G      1.153     0.4962     0.2515        790        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.2it/s 0.2s.3s
                   all         93       2846      0.543      0.481       0.44      0.155
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.2it/s 0.2s
                   all         93       2846      0.543      0.481       0.44      0.155

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/200      6.23G      0.945     0.5119      0.184        431        256: 5% ╸─────────── 1/20 1.9it/s 0.3s<9.8s

...

      9/200      7.67G      0.829     0.4841     0.1469        410        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
      9/200      7.67G      0.829     0.4841     0.1469        410        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.1it/s 0.2s.3s
                   all         93       2846      0.739      0.686      0.643      0.284
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.1it/s 0.2s
                   all         93       2846      0.739      0.686      0.643      0.284

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     10/200      7.76G     0.9185     0.4579     0.1995        601        256: 5% ╸─────────── 1/20 1.8it/s 0.4s<10.4s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     10/200      7.76G     0.8729      0.477     0.1714        420        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
     10/200      7.76G     0.8729      0.477     0.1714        420        256: 100% ━━━━━━━━━━━━ 20/20 5.9it/s 3.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.2it/s 0.2s.3s
                   all         93       2846      0.708       0.65      0.613      0.244
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.2it/s 0.2s
                   all         93       2846      0.708       0.65      0.613      0.244

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     11/200      7.76G     0.7884     0.4994     0.1443        462        256: 5% ╸─────────── 1/20 2.0it/s 0.3s<9.3s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     11/200      7.76G     0.8192     0.4921     0.1516        670        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
     11/200      7.76G     0.8192     0.4921     0.1516        670        256: 100% ━━━━━━━━━━━━ 20/20 6.1it/s 3.3s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.8it/s 0.2s.3s
                   all         93       2846     0.0394     0.0474     0.0111    0.00169
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 12.8it/s 0.2s
                   all         93       2846     0.0394     0.0474     0.0111    0.00169


## Results

In [ ]:
import json 

df = pd.DataFrame(results)

summary = {
    'exp_id': exp_id,
    'seed': SEED,
    'epochs': EPOCHS,
    'img_size': IMG_SIZE,
    'batch': BATCH,
    'patience': PATIENCE,
    'models': [m['name'] for m in MODELS],
    'results': results,
    'best_model': results[df['test_map50'].idxmax()]['model'] if len(results) > 0 else None,
    'best_test_map50': float(df['test_map50'].max()) if len(results) > 0 else 0.0
}

exp_results_dir = RESULTS_DIR / exp_id
exp_results_dir.mkdir(exist_ok=True)

with open(exp_results_dir / "summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print("\nResults")
print(df[['model', 'val_f1', 'val_map50', 'test_f1', 'test_map50']].to_string(index=False))
print(f"\nBest: {summary['best_model']} (test_map50={summary['best_test_map50']:.4f})")
print(f"Saved: results/{exp_id}/summary.json")